In [31]:
import tensorflow as tf
import numpy as np

from pinn import PINNModel, model1
from accuracy import AccuracyCalc
from plotting import NNPlots
from data_gen import DataGenerator

In [32]:
def real_u(area):
    area = np.array(area)
    if len(area.shape) >= 3:
        x = area[0]
        y = area[1]
        return np.sin(np.pi * x) * np.sin(np.pi * y)
    else:
        return np.array([np.sin(np.pi * x) * np.sin(np.pi * y)  for x, y in area])

In [33]:
# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [34]:
area_grid_size = (50, 50)
x_limits = (0, 2)
y_limits = (0, 2)
border_grid_size = (50, 50)

EPOCHS = 6000
EPRINT = 500
LOSS = 1e-3 # 1e-4

model = PINNModel(model1(), optm = tf.keras.optimizers.Adam(learning_rate=0.001))

dataGen = DataGenerator(x_limits, y_limits, model.predict, real_u)

# info = Task(model.predict, real_u, (x_limits, y_limits), area_grid_size, border_grid_size)

In [ ]:
# model.load_weights("../models/model0.weights.h5")
model.train(1, dataGen.inner_pairs(area_grid_size), dataGen.border_pairs(border_grid_size), EPOCHS, LOSS, EPRINT)
# model.save("../models/model0.weights.h5")

In [ ]:
acc = AccuracyCalc(dataGen, (50, 50))
print("Error:", acc.mse())
print("Error:", acc.maxe())

In [37]:
model.load_weights("../models/model0.weights.h5")
# model.train(1, dataGen.inner_pairs(area_grid_size), dataGen.border_pairs(border_grid_size), 500, LOSS, EPRINT)
# model.save("../models/model1.weights.h5")

In [ ]:
dataGen._predict = model.predict
acc = AccuracyCalc(dataGen, (50, 50))
print("Error:", acc.mse())
print("Error:", acc.maxe())

In [ ]:
# plotter = NNPlots((x_limits, y_limits), area_grid_size, model.predict, real_u, info._predict_val)
plotter = NNPlots(dataGen, (200, 200))

In [ ]:
plotter.plot2d_contour(color_map="gnuplot2")